In [1]:
#
#
#Author: WU Nan
#Date: 2016.12.5
#Description: This is the csv to csv programme file to 
#Further target: ⚠️
#To make 5 star for a stock on risk assessment: ND:要细分成四档；DD细分成6档。 0.5*6+0.5*4最多是5颗星。
#
#
#
from __future__ import print_function
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import sys
from operator import add
from pyspark import SparkContext

import numpy as np
import csv
import math
from scipy import stats
import pandas as pd
import scipy


#period: the days of period you choose
#✅WELL DONE
def get_csv_data(filename):
    filename = "file:/Users/nancywu/sparkhadoop/datatest/Stock/"+filename+".csv"
    File = sc.textFile(filename)
    File.map(lambda line: line.split(","))
    File.filter(lambda line: len(line) > 0)
    File.map(lambda line: (line[0], line[1]))
    data = File.collect()
    stock_text = [d.split(",") for d in data]

    #From backward of Period of present date (90 days before 2016.4.1) to present date 2016.4.1
    #historical period you want to choose
    period = 365*3

    open_price = [round(float(stock_text[i][1]),10) for i in range(period,0,-1)]
    close_price = [round(float(stock_text[i][4]),10) for i in range(period,0,-1)]
    volume=[long(stock_text[i][5]) for i in range (period, 0,-1)]
    date=["Date"]
    [date.append(stock_text[i][0]) for i in range(period,0,-1)]

    print ("get_csv_data: done")
    return open_price,close_price,volume,date


def normal_distribution_test(open_price,close_price):
        ##risk distance round精度调整
            risk_assessment_initial = list(map(lambda x: round((x[0]-x[1]),2), zip(open_price, close_price)))
            ##
            print("====risk assessment initial======")
            tmp=sc.parallelize(risk_assessment_initial)
            Change_initial = tmp.map(lambda distance: (distance, 1))\
                    .reduceByKey(lambda a, b: a + b)
            Change_list=Change_initial.collect()
            distance_list=[]
            frequency_list=[]
            for (distance,frequency) in Change_list:
                #这里限制了波动噪声。
                if (frequency<len(risk_assessment_initial)*0.5):
                    distance_list.append(distance)
                    frequency_list.append(frequency)
                else:
                    Change_list.remove([distance,frequency])
        #3 methods of statistics to test how distribution it is.
        #Make risk range for particular stock
            x = frequency_list
            shapiro_results = scipy.stats.shapiro(x)
            a1=shapiro_results[0]
            a2=round(shapiro_results[1],8)
            matrix_sw = [['', 'DF', 'Test Statistic_SW', 'p-value'],['Sample Data', len(x) - 1, a1, a2]]
            print(matrix_sw)
            #shapiro_result more close to 1 more probabily of normal distribution it is
            #####
            ks_results = scipy.stats.kstest(x, cdf='norm')
            b1=ks_results[0]
            b2=round(ks_results[1],8)
            matrix_ks = [['', 'DF', 'Test Statistic_KS', 'p-value'],['Sample Data', len(x) - 1, b1, b2]]
            print(matrix_ks)
            ###
            dagostino_results = scipy.stats.mstats.normaltest(x)
            c1=dagostino_results[0]
            c2=round(dagostino_results[1],8)
            matrix_dp = [['', 'DF', 'Test Statistic_dp', 'p-value'],['Sample Data', len(x) - 1, c1, c2]]
            print(matrix_dp)
        ###########risk_ND_signal##############
            if (abs(a1-a2)>0.8 and (abs(b1-b2)>0.8 and b2<0.0005) and (abs(c1-c2)>0.8 and c2<0.0005)):
                #normal distribution. low risk
                signal=3
            elif (abs(a1-a2)>1 or abs(b1-b2)>1 or abs(c1-c2)>1): 
                signal=2
            else:
                #not normal distribution. high risk
                signal =1
            return signal,Change_list
    
    
def max_drawdown_test(timeseries):
    # 回撤结束时间点
        i = np.argmax(np.maximum.accumulate(timeseries) - timeseries)
        #timeseries是需要探测的最大回撤率的时间范围，timeseries=[price1,price2,price3],时间标度:day_i,day_i+1
        # 回撤开始的时间点
        j = np.argmax(timeseries[:i])
        #print i,j
        dd_return =(float(timeseries[i]) / timeseries[j]) - 1
        ####回撤signal_setting##########
        if (abs(dd_return)<0.03):
            risk_DD_signal=5
        elif (abs (dd_return)<0.1):
            risk_DD_signal=4
        elif (abs (dd_return)<0.3):
            risk_DD_signal=3
        elif (abs (dd_return)<0.8):
            risk_DD_signal=2
        else:
            risk_DD_signal=1
        return risk_DD_signal, dd_return

def risk_assessment(filename):
        open_price,close_price, volume, date= get_csv_data(filename)
        risk_ND_signal, Change_list=normal_distribution_test(open_price,close_price)
        risk_DD_signal, dd_return = max_drawdown_test(close_price)

        print (filename, "dd_return is:",dd_return)
        return Change_list,risk_DD_signal, risk_ND_signal

def writeToElastic(fileindex,es,filename,risk_ND_text,risk_rate):
    df=risk_ND_text
    j = 1
    actions = []
    count = int(len(df))
    while (j < count):
        action = {
                   "_index": fileindex, # 这里不可以是大写，都是小写
                   "_type": filename,
                   "_id": j,
                   "_source": {
                               "pricechange":df[j][0],
                               "frequency":float(df[j][1]),
                               "riskrate":risk_rate
                                }
                   }
        print(action)
        actions.append(action)
        j += 1
        if (len(actions) == 180):
            helpers.bulk(es, actions)
            del actions[0:len(actions)]
            
    if (len(actions) >0 ):
            helpers.bulk(es, actions)
            del actions[0:len(actions)]
    
        
if __name__ =="__main__":
    #sc = SparkContext(appName="Monte Carlo")
    Ticker = sc.textFile("file:/Users/nancywu/sparkhadoop/datatest/Ticker_50.csv")
    filelist = Ticker.map(lambda f: f.split(",")).collect()
    print(filelist)
    es = Elasticsearch(['http://127.0.0.1:9200'])
    #####Risk rate preparation
    ####Risk level set up by Customer set 
    ###5:100%; 4:80%; 3:60%; 2,1,0
    risk_list=[]
    risk_w_ND=0.5
    risk_w_DD=0.5
    ######risk_w_ND+risk_w_DD+...+other risk_w_assess=100%
    ######
    print("===========start============")
    for f in filelist:
        try:
            name = f[0]+".csv"
            print (name,"simulation",f[0])
            Change_list_Normal_Distribution, risk_DD_signal, risk_ND_signal = risk_assessment(f[0])
            print("=====risk assessment pass=====")
            risk_rate=risk_DD_signal*risk_w_DD  + risk_ND_signal*risk_w_ND
            risk_list.append([f[0],risk_rate])
            print(Change_list_Normal_Distribution)
            ########
            print("=======output pass=======")
            #sc.parallelize(Change_list_Normal_Distribution).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + 'risk_change_list'+name)
            writeToElastic("riskfreq",es,name,Change_list_Normal_Distribution,risk_rate)
            print("=======ES pass=======")
        except:
            import traceback
            traceback.print_exc()
            print("No service for this stock")
    #sc.parallelize(risk_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/stock_risk_rate_list")




[[u'A0E1.BE'], [u'A0U1.F'], [u'A1G.DE'], [u'A1G.F'], [u'A1H.SG'], [u'A2A.MI'], [u'A2H.BE'], [u'A34.F'], [u'A3M.MC'], [u'A50.SI'], [u'A5F.SG'], [u'A60.F'], [u'A60.HM'], [u'A60.MU'], [u'A60.SG'], [u'A7Z.F'], [u'A8B.BE'], [u'A8B.DE'], [u'AA'], [u'AA.MX'], [u'AA2.BE'], [u'AAA.L'], [u'AAB.TO'], [u'AAC'], [u'AAC.V'], [u'AACAY'], [u'AAD.DE'], [u'AAEV.L'], [u'AAFA.F'], [u'AAGIY'], [u'AAH3.BE'], [u'AAH3.DU'], [u'AAHTF'], [u'AAK.F'], [u'AAL'], [u'AAL.L'], [u'AALI.JK'], [u'AAM.F'], [u'AAN'], [u'AAOI'], [u'AAR-UN.TO'], [u'AATG.L'], [u'AAU'], [u'AAU.SG'], [u'AAV.TO'], [u'AAVC.L'], [u'AAWC'], [u'AAX.AX'], [u'AAXT'], [u'AB'], [u'AB3A.DE'], [u'ABAC'], [u'ABAN.BO'], [u'ABAN.NS'], [u'ABAX'], [u'ABB'], [u'ABBY'], [u'ABCA.PA'], [u'ABCB'], [u'ABCO'], [u'ABCP'], [u'ABD.L'], [u'ABDTF'], [u'ABEOW'], [u'ABEV3.SA'], [u'ABF.L']]
===========start============
A0E1.BE.csv simulation A0E1.BE
No service for this stock

Traceback (most recent call last):
  File "<ipython-input-1-6734e1571b9f>", line 177, in <module>
    Change_list_Normal_Distribution, risk_DD_signal, risk_ND_signal = risk_assessment(f[0])
  File "<ipython-input-1-6734e1571b9f>", line 123, in risk_assessment
    open_price,close_price, volume, date= get_csv_data(filename)
  File "<ipython-input-1-6734e1571b9f>", line 41, in get_csv_data
    open_price = [round(float(stock_text[i][1]),10) for i in range(period,0,-1)]
IndexError: list index out of range
Traceback (most recent call last):



A0U1.F.csv simulation A0U1.F
No service for this stock

  File "<ipython-input-1-6734e1571b9f>", line 177, in <module>
    Change_list_Normal_Distribution, risk_DD_signal, risk_ND_signal = risk_assessment(f[0])
  File "<ipython-input-1-6734e1571b9f>", line 123, in risk_assessment
    open_price,close_price, volume, date= get_csv_data(filename)
  File "<ipython-input-1-6734e1571b9f>", line 41, in get_csv_data
    open_price = [round(float(stock_text[i][1]),10) for i in range(period,0,-1)]
IndexError: list index out of range
Traceback (most recent call last):



A1G.DE.csv simulation A1G.DE
No service for this stock

...

  File "<ipython-input-1-6734e1571b9f>", line 177, in <module>
    Change_list_Normal_Distribution, risk_DD_signal, risk_ND_signal = risk_assessment(f[0])
  File "<ipython-input-1-6734e1571b9f>", line 124, in risk_assessment
    risk_ND_signal, Change_list=normal_distribution_test(open_price,close_price)
  File "<ipython-input-1-6734e1571b9f>", line 67, in normal_distribution_test
    Change_list.remove([distance,frequency])
ValueError: list.remove(x): x not in list
Traceback (most recent call last):



ABDTF.csv simulation ABDTF
No service for this stock

  File "<ipython-input-1-6734e1571b9f>", line 177, in <module>
    Change_list_Normal_Distribution, risk_DD_signal, risk_ND_signal = risk_assessment(f[0])
  File "<ipython-input-1-6734e1571b9f>", line 123, in risk_assessment
    open_price,close_price, volume, date= get_csv_data(filename)
  File "<ipython-input-1-6734e1571b9f>", line 41, in get_csv_data
    open_price = [round(float(stock_text[i][1]),10) for i in range(period,0,-1)]
IndexError: list index out of range
Traceback (most recent call last):



ABEOW.csv simulation ABEOW
No service for this stock
ABEV3.SA.csv simulation ABEV3.SA
get_csv_data: done
====risk assessment initial======
[['', 'DF', 'Test Statistic_SW', 'p-value'], ['Sample Data', 362, 0.3609151244163513, 0.0]]
[['', 'DF', 'Test Statistic_KS', 'p-value'], ['Sample Data', 362, 0.84134474606854293, 0.0]]
[['', 'DF', 'Test Statistic_dp', 'p-value'], ['Sample Data', 362, 644.69442471854961, 0.0]]
ABEV3.SA dd_return is: -0.966637584298
=====risk assessment pass=====
[(1.25, 4), (1.5, 3), (4.0, 2), (-5.25, 3), (-3.9, 1), (8.0, 1), (-3.15, 1), (0.19, 11), (-2.65, 1), (-2.9, 1), (5.5, 1), (-0.5, 8), (0.13, 11), (7.5, 1), (7.05, 2), (-13.5, 1), (-1.5, 3), (-7.3, 1), (7.75, 1), (-4.8, 2), (-1.25, 2), (0.3, 12), (0.21, 4), (-8.85, 1), (0.46, 4), (-10.25, 1), (-2.5, 4), (0.85, 3), (3.75, 4), (-29.45, 1), (-0.59, 1), (0.38, 1), (-4.55, 2), (-6.5, 3), (1.45, 2), (-0.15, 13), (-0.34, 2), (-3.65, 1), (0.42, 1), (0.0, 80), (-9.75, 1), (3.65, 2), (-0.23, 8), (-5.05, 1), (-7.5, 1), (

  File "<ipython-input-1-6734e1571b9f>", line 177, in <module>
    Change_list_Normal_Distribution, risk_DD_signal, risk_ND_signal = risk_assessment(f[0])
  File "<ipython-input-1-6734e1571b9f>", line 123, in risk_assessment
    open_price,close_price, volume, date= get_csv_data(filename)
  File "<ipython-input-1-6734e1571b9f>", line 41, in get_csv_data
    open_price = [round(float(stock_text[i][1]),10) for i in range(period,0,-1)]
IndexError: list index out of range


In [ ]:
x=[1,1,1,1]
y=[2,2,2,2]
alist=[["distance","freq"],x,y]
blist=pd.read_csv('file:/Users/nancywu/sparkhadoop/datatest/largetest/AA.csv')
alist=pd.DataFrame(alist,columns=['AA','FREQ','D','DD'])
print (alist)

 #pandaslist=pd.DataFrame(change_list,columns=['PriceChange','Frequence'])
            #print(pandaslist.PriceChange)

In [ ]:
x=[[1,2],[22,1]]
#del(x([1,2])
x.remove([1,2])
print (x)